## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd #импорт библиотеки для чтения файла и загрузки данных в проект
data = pd.read_csv('/datasets/data.csv') #читаем таблицу
data.info() #смотрим, из чего таблица состоит

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

С точки зрения корректности определения типов данных их полноты, качество выгрузки неплохое. В первые несколько минут изучения можно сделать следующие выводы: 

1. Визуально в таблице 5 столбцов, содержащих явно текстовые данные и метод info(), примененный к полученному датафрейму говорит об этом же => заполнение столбцов типами корректное;
2. В срезе 21525 наблюдений и 10/12 столбцов имеют такое же количество записей. Так как тип данных по столбцам определился корректно, можно предположить, что во всех строках столбцов указаны более-менее приближенные к действительности значения;
3. В таблице 2 столбца, в которых менее 21525 наблюдений: days_emloyed и total_income. Вероятно также важен факт точного соответствия количества записей в этих столбцах;
4. Визуальный просмотр файла также показывает, что текстовая запись категорийных значений (образование, семейный статус и т.п.) сделана в разных регистрах, хотя идентификаторы корректные. Вероятно, на уровне источника данных нет проверки данных при заполнении, либо заполнения текстового значения из отдельного справочника вида "ID-Категория". 

Первичные гипотезы:

1. Между значением количества дней трудового стажа и общего дохода, вероятно, есть взаимосвязь. По некой неустановленной причине люди не хотят или не могут заполнять эти значения - стоит посмотреть, что объединяет клиентов в этой группе;
2. Стоит проверить корректность заполнения идентификаторов и привести текстовые значения категорий в единообразное состояние;

### Шаг 2. Предобработка данных

### Обработка пропусков

При первичном изучении среза мы увидели, что в столбцах 'days_employed' и 'total_income' столбцов меньше, чем в остальных. Это значит, что в них содержатся пропуски. 

Мы видим, что в каждом из столбцов 19351 заполненное значение, то есть по 2174 пропущенных значения. Для проверки гипотезы о наличии некой взаимосвязи между значениями, необходимо убедиться, что строк с пропущенными значениями не более 2174. 

In [2]:
#при помощи логической индексации мы отбираем строки у которых не заполнены значения в столбцах 'days_employed'
#и 'total_income', считаем количество строк в полученной выборке и выводим на экран число
print(len(data.loc[(data['days_employed'].isnull()) & (data['total_income'].isnull())]))



2174


Мы видим, что пропущенные значения пропущены одновременно для каждого клиента (если не указан стаж, то не указан и общий доход). Таким объемом данных мы не можем пренедречь, так как строки с пропусками - это примерно 10% от общего количества наблюдений. 

На верхнем уровне причины отсутствия данных можно разделить на две группы: 

1. Случайные (забыли заполнить и система приняла форму без заполненных данных, данные о размере дохода и стаже импортируются из внешней системы и потерялись в процессе)
2. Намеренные (клиент не имеет официального места работы, поэтому у него нет стажа, а доход не указал из соображений скрытности)

Пропущенные значения необходимо заполнить. Начнем с заполнения столбца 'days_employed'. Заполнить его необходимо следующим образом:

1. Проанализировать заполненные значения столбца на предмет корректности;
2. Проверить данные в столбце 'dob_years';
3. Разделить клиентов на возрастные категории, так как между стажем и возрастом есть явная корреляция;
4. Для каждой категории вычислить среднее значение и использовать его для строк, возраст клиентов в которых соответствует той или иной рассчитанной нами возрастной группе. 

Перейдем к проверке:

Визуальный осмотр таблицы показал: в столбце 'days_employed' присутствуют отрицательные значения.
Для расчета среднего или медианного значения нам необходимо от них избавиться. 

Оценим их количество: 

In [3]:
print(len(data.loc[data['days_employed'] < 0])) #отбираем строки, в которых значение дней трудового стажа отрицательно и считаем их количество.

15906


Более 70% записей таблицы имеют отрицательные значения в количестве дней стажа. Вероятно, проблема изменения знака массовая и как-то связана с экспортом данных, либо методикой расчета.

Гипотеза: так как стаж - это разность между текущей датой и датой начала трудовой деятельности, то можно предположить, что при расчете значения был нарушен порядок вычислений. Для нормализации значений достаточно для всех значений использовать модуль числа. 

Для этого мы напишем функцию и применим ее к таблице:

In [4]:
def modul_transition(row):
    days_employed = row['days_employed']
    if days_employed < 0: #если значение дней трудового стажа отрицательное, то необходимо умножить его на -1 и вернуть в строку
        return days_employed * -1
    if days_employed == 0 or days_employed > 0: #если значение нулевое, либо более 0, то просто умножаем на 1
        return days_employed * 1

data['days_employed'] = data.apply(modul_transition, axis=1) #применяем функцию к таблице
print(len(data.loc[data['days_employed'] < 0])) #повторно проверяем таблицу на отрицательные значения

0


Теперь, когда отрицательные значения преобразованы в положительные, необходимо оценить корретность данных в столбце с данными о трудовом стаже. Наиболее простым способом оценки корректности данных о продолжительности трудового стажа будет сравнение его с возрастом потенциально клиента. Так как возраст клиента указан в годах, то мы произведем расчета стажа в годах и добавим его в виде отдельного столбца в таблицу - 'years_employed':

In [5]:
data['years_employed'] = data['days_employed'] / 365 #добавляем новый вычисляемый столбец с данными путем деления дней трудового стажа на 365 дней
#print(data.head()) #оцениваем полученный результат

В 5 строке у одного из клиентов трудовой стаж в годах превышает 932 года, что явно говорит нам о какой-то аномалии в данных о трудовом стаже. Предположим, что минимальный возраст от которого возможно отсчитывать трудовой стаж - 14 лет. Тогда нам необходимо найти самого пожилого клиента в нашем списке и использовать разность между его возрастом и минимальным возрастом начала трудового стажа в качестве критерия для отбора диапазона строк с некорректным значением трудового стажа. 

In [6]:
maximum_yrs_employed = data['dob_years'].max() - 14 #ограничиваем максимальную продолжительность трудового стажа
incorrect_days_employed = data.loc[data['years_employed'] > maximum_yrs_employed] #отбираем строки, в которых количество лет трудового стажа превышает максимальное значение
#print(incorrect_days_employed.sort_values('years_employed').head(10)) #выводим отсортированный результат для оценки
incorrect_days_employed_min = incorrect_days_employed['days_employed'].min() #фиксируем минимальное значение дней стажа
incorrect_days_employed_max = incorrect_days_employed['days_employed'].max() #фиксируем максимальное значение дней стажа
print()
print(incorrect_days_employed_min)
print(incorrect_days_employed_max)


328728.72060451825
401755.40047533


Мы видим, что некорректный стаж варьируется в диапазоне 900-1100 лет. Две наиболее очевидные причины ошибки:

1. Неправильно введен, либо не введен год при заполнении персональных данных клиента;
2. Информация о годе начала стажа клиента была потеряна при экспорте данных.

Так данные о трудовом стаже в 900+ лет нам не могут быть полезны, для вычисления среднего значения количества дней стажа мы отберем строки с корректными значениями. Так как количество трудового стажа связано с возрастом клиента, сгруппируем их в 4 группы по возрасту: до 20 лет, 20-40 лет, 40-60 лет, старше 60 лет. Для удобства создадим 4 переменные, в которые запишем средние значения трудового стажа для каждой группы: 

In [7]:
up_to_20_days_employed_mean = data.loc[(data['days_employed'] < incorrect_days_employed_min) & (data['dob_years'] <= 20), 'days_employed'].mean()
f_20_to_40_days_employed_mean = data.loc[(data['days_employed'] < incorrect_days_employed_min) & (data['dob_years'] > 20) & (data['dob_years'] <= 40), 'days_employed'].mean()
f_40_to_60_days_employed_mean = data.loc[(data['days_employed'] < incorrect_days_employed_min) & (data['dob_years'] > 40) & (data['dob_years'] <= 60), 'days_employed'].mean()
f_60_days_employed_mean = data.loc[(data['days_employed'] < incorrect_days_employed_min) & (data['dob_years'] > 60), 'days_employed'].mean()

#У нас есть 4 средних значения и мы можем их использовать для заполнения пропуском и некорректных значений
#Напишем функцию, которая будет брать строку в датафрейме, проверять значение в столбце 'days_employed' 
#И если значения нет, либо оно равно или больше ранее определенной переменной 'incorrect_days_employed_min'
#Мы будем записывать в дни стажа расчетное среднее значение

data.loc[data['days_employed'].isnull() == True, 'days_employed'] = 0 #заполняем пустые значения нулями для простоты работы функции

def days_employed_check(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    if days_employed == 0:
        if dob_years <= 20:
            return up_to_20_days_employed_mean
        if dob_years <= 40: 
            return f_20_to_40_days_employed_mean
        if dob_years <= 60: 
            return f_40_to_60_days_employed_mean
        return f_60_days_employed_mean
    if days_employed >= incorrect_days_employed_min:
        if dob_years <= 20:
            return up_to_20_days_employed_mean
        if dob_years <= 40: 
            return f_20_to_40_days_employed_mean
        if dob_years <= 60: 
            return f_40_to_60_days_employed_mean
        return f_60_days_employed_mean
    if days_employed < incorrect_days_employed_min and days_employed != 0:
        return days_employed
        
data['days_employed'] = data.apply(days_employed_check, axis=1) #применяем функцию к датафрейму
del data['years_employed'] #удаляем добавленный ранее вычисляемый столбец 'years_employed', так как он нам больше не нужен

print(data.info()) #оцениваем результат и видим, что в столбце 'days_employed' все столбцы заполнены
#print(data.head()) #визуально столбец 'days_employed' также выглядит более корректно, чем ранее

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


У нас остались пропуски в столбце 'total_income', что видно на шаге выше. 
Метод заполнения данных в этом столбце схож со стажем, но имеет смысл использовать иные значения:
1. Вместо среднего арифметического более разумно использовать медиану, так как колебания уровня дохода могут быть значительно выше таковых для стажа
2. Для поиска более точного значения также имеет смысл сгруппировать клиентов по определенному признаку, но при оценке уровня дохода лучше использовать тип дохода, то есть данные столбца 'income_type'
3. Так как 'income_type' явно категорийная переменная следует провести оценку качества данных в столбце перед переходом к заполнению пропусков в ежемесячных доходах.

In [8]:
print(data['income_type'].value_counts()) #мы смотрим на количество записей по каждом типу дохода, а заодно на качество отражения самих типов
income_type_median_values = data.groupby('income_type') #группируем дата-сет по типу дохода и записываем в отдельную переменную

#кладем в переменную series со медианным уровнем дохода по каждому типу
#так мы можем легко получить доступ к медианному значению для заполнения пропусков по доходу:
income_type_median_values = income_type_median_values['total_income'].median() 
#print(income_type_median_values) #можем визуально оценить значения среднего дохода по каждой категории

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Теперь нам необходимо заполнить пропуски в значениях ежемесячного дохода для клиентов с незаполненными значениями. Для этого напишем функцию, которая проверит весь столбец 'total_income' в таблице и, в случае отсутствия в нем значений, заполнит медианным значением для устранения пропусков.  

In [9]:
def income_filler(row):
    income_type = row['income_type']
    total_income = row['total_income']
    if total_income > 0: #если значение total_income больше нуля (то есть, заполнено), оставляем старое значение
        return total_income
    total_income_median = income_type_median_values[income_type] #то объявляем переменную, равную соответствующему значения из сформированного ранее списка медианных значений по категориям
    return total_income_median #возвращаем медианное значение вместо пустоты
        
data['total_income'] = data.apply(income_filler, axis=1) #применяем функцию к дата-сету
print(data.info()) #оцениваем результат

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Теперь в таблице нет пропусков. 

### Вывод

При первоначальном изучении среза были обнаружены два столбца с пропусками. Путем подсчета среднего и медианного значений с категоризацией для разных столбцов было произведено заполнение пропусков. 

### Замена типа данных

In [10]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


В таблице 2 столбца, тип данных которых - вещественное число. Вероятно, при экспорте данных из системы, значения в столбце количества дней трудового стажа были вычислены относительно конкретного момента выгрузки, поэтому в таблице мы можем наблюдать довольно много знаков после запятой. Для работы с данными о днях стажа в нашем случае такая точность не требуется, поэтому мы просто изменим тип данных в столбце на целое число. 

По схожей причине мы преобразуем среднемесячный уровень дохода в целое число, так как с такими данными проще работать, а копейки для достижения целей исследования не требуются. 

In [11]:
data['days_employed'] = data['days_employed'].astype('int') #при помощи метода astype() мы преобразуем значения в тот тип, который передаем методу в параметрах
data['total_income'] = data['total_income'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB
None


### Вывод

При помощи метода astype() мы преобразовали вещественные числа в целые, что в целом не снижает потенциальную точность ответа на поставленные перед исследованием вопросы, но упрощает работу с данными. 

### Обработка дубликатов

В таблице присутствовали 54 полных дубликата, которые удалось выявить при помощи метода duplicated(). Методом drop_duplicates() эти дубликаты мы убрали из таблицы. 
В соответствии с изначальной гипотезой, на факт погашения крелита в срок влияют два фактора: семейное положение и количество детей. При помощи аргумента value_counts() мы оценили количество значений в столбцах 'family_status' и 'family_status_id': количество уникальных значений и количество наблюдений для каждого идентичны, поэтому столбец 'family_status' не нуждается в удалении дубликатов. 

Вероятная причина появления полных дубликатов - дублирование информации об обращении клиентов. Это может обусловлено особенностями хранения данных, либо спецификой процесса извлечения данных из учетной системы. 

Также в столбце 'education' присутствуют различные варианты записи стандартных категорий. Использование метода 'unique' в комбинации с функцией len возвращает информацию о 15 уникальных значениях. В то время как аналогичная комбинация, примененная к столбцу 'education_id', говорит лишь о 5 уникальных идентификаторах. Визуальная оценка диапазона уникальных значений образования говорит нам о проблем с регистром символов: вероятно, произошла какая-то ошибка работы с данными, так как отсутствие проверки данных на этапе ввода привело бы к значетельно большему количеству ошибок. При помощи метода str.lower() мы приводим значения в столбце к нижнему регистру и повторная проверка говорит о соответствии количества уникальных значений столбцов 'education' и 'education_id'.

In [12]:
data = data.drop_duplicates().reset_index(drop=True)
data['education'] = data['education'].str.lower()

### Вывод

В данных присутствовали полные дубликаты строк, а также дубликаты категорийных значений, полученные в результате использования разного регистра для символов. Стандартными методами Python эти дубликаты удалось устранить. 

### Лемматизация

In [13]:
from pymystem3 import Mystem #импортируем библиотеку pymystem и записываем ее в переменную m для обращения к ее функциям и методам
m = Mystem()

def lemmatization(row):
    purpose = row['purpose'] #берем str с целью получения кредита в строке
    purpose = m.lemmatize(purpose) #лемматизируем слова в цели кредита по текущей строке
    purpose = ''.join(purpose).rstrip('\r\n') #склеиваем полученный список в строку и удаляем символ переноса строки
    return purpose #возвращаем лемматизированную строку на место данных, которые мы лемматизировали
    
data['purpose'] = data.apply(lemmatization, axis=1) #применяем функцию к дата-сету 

In [14]:
real_goals = set() #объявляем переменную в которую запишем все уникальные короткие значения в колонке

for purpose in data['purpose']: #при помощи цикла пробегаемся по столбцу и в множество пишем значения всех строк, которые не превышают 1 слово
    if len(purpose.split()) == 1:
        real_goals.add(purpose)
    else: 
        continue

#в результате работы цикла мы получили множество с 5 уникальными значениями = целями получения кредита
# {'свадьба', 'недвижимость', 'автомобиль', 'жилье', 'образование'}
   
    
#функция проверят, есть ли значения из множества уникальные целей в 1 слово в строке
#если есть, меняет значение purpose в строке на 1 существительное

def single_word_goal_modifier(row): 
    purpose = row['purpose']
    purpose = purpose.split()
    for goal in real_goals:
        if goal in purpose:
            purpose = goal
    return purpose

data['purpose'] = data.apply(single_word_goal_modifier, axis=1)

In [15]:
print(data['purpose'].value_counts()) #все строки дата-сета сгруппированы по 5 понятным категориям

недвижимость    6353
жилье           4461
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose, dtype: int64


### Вывод

При помощи библиотеки pymystem и нескольких функций нам удалось извлечь леммы из изначальных формулировок целей получения кредита, а также редуцировать значения в каждой строке до 5 понятных категорий, по которым впоследствии будет нетрудно определить долю возвращающих кредит в срок. 

### Категоризация данных

Ранее, в рамках процесса по заполнению пропусков в данных, была проведена категоризация данных: мы распределили клиентов по возрастным группам для рассчета среднего значения трудового стажа в каждой возрастной группе. Такой подход наиболее очевидный, так как между стажем и возрастом клиента есть зависимость. Категоризация была произведена при помощи фукнции для одной строки. 

Также данных присутствует столбец, в котором, несмотря на численное значение, отражается категорийная переменная - это количество детей. 

In [16]:
print(data['children'].value_counts()) #смотрим на количество уникальных значений по столбцу и количество записей для каждого значения

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Гипотеза исследования: количество детей и семейное положение влияют на факт погашения кредита в срок. 

Таким образом, обработку среза начинаем с нормализации данных в столбце 'children'. 
Мы видим, что в нашем списке 8 уникальных значений. Из них 2 явно выделяются: -1 и 20. 
Мы не можем использовать значение -1 для характеристики количества детей, так как оно не является натуральным. В нашей выборке есть два приближенных значения: 0 и 1. Так как физически сложно ввести 1 вместо 0, мы остановимся на гипотезе о том, что знак минус появился у единицы случайно в результате выгрузки, либо обработки данных в источнике и изменим значение -1 на 1. 

Значение 20 также не выглядит реалистичным по следующим причинам: 

1. Между значениями 5 и 20 отсутствуют промежуточные значения, что очень странно в такой, достаточно многочисленной, выборке;
2. Количество записей со значением 20 больше, чем суммарное количество записей со значениями 4 и 5 детей. На уровне личностных ощущений предположу, что такой вариант развития событий нереалистчен. 

Вероятно, при вводе данных была допущена ошибка и вместо значения 2 было введено 20, либо ошибочна была введена двойка и клиент ввел ноль, не удалив предыдущее значение. Так как в группе '2 детей' дополнительные 76 записей составят порядка 4% от общей суммы, мы изменим значение 20 на 0, так как это в наименьшей степени повлияет на точность итоговых выводов. 

In [17]:
data.loc[data['children'] <0, 'children'] = 1 #отбираем строки со значением <0 по количеству детей и изменяем значение в них на 1
data.loc[data['children'] == 20, 'children'] = 0 #отбираем строки со значением '20' и изменяем его на '0'
print(data['children'].value_counts())

0    14183
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


### Вывод

Данные были распределены по категориям в процессе удаления пропусков. Кроме того, восстановлена корректная категоризация по количеству детей, что важно в контексте поставленного перед исследованием вопроса. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [18]:
children_grouped = data.groupby('children') #создаем Series, в который группируем дата-сете по количеству детей
print(children_grouped['debt'].mean().sort_values()) #выводим среднее значение реквизита debt для каждой категории
print(children_grouped['debt'].value_counts())

children
5    0.000000
0    0.075513
3    0.081818
1    0.091639
2    0.094542
4    0.097561
Name: debt, dtype: float64
children  debt
0         0       13112
          1        1071
1         0        4411
          1         445
2         0        1858
          1         194
3         0         303
          1          27
4         0          37
          1           4
5         0           9
Name: debt, dtype: int64


### Вывод

Судя по среднему значению булевого параметра debt, с ростом количества детей в семье растет и среднее значение по параметру "были ли задолженности по кредитам". Также мы видим, что клиенты с 5 детьми имеют среднюю оценку по этому параметру = 0. Однако эти данные нельзя использовать для выводов, так как выборка в 9 человек в данном случае нерепрезентативна.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
family_status_grouped = data.groupby('family_status')
print(family_status_grouped['debt'].mean().sort_values())
print(family_status_grouped['debt'].value_counts())

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075421
гражданский брак         0.093202
Не женат / не замужем    0.097509
Name: debt, dtype: float64
family_status          debt
Не женат / не замужем  0        2536
                       1         274
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3775
                       1         388
женат / замужем        0       11413
                       1         931
Name: debt, dtype: int64


### Вывод

Мы видим, что наименьший средний балл по параметру задолженностей был у группы 'вдовец / вдова', однако эта группа не слишком многочисленна, поэтому посмотрим на другие группы. Здесь мы видим, что даже гражданский брак незначительно влияет на ответственность заемщиков. Но наиболее очевидные результаты по группам женатых и разведенных: средний балл значительно ниже, что приводит нас к заключению о влиянии семейного статуса на задолженность. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [20]:
income_grouped = data.groupby(['debt', 'income_type']).agg({'total_income': 'mean'})
print(income_grouped)
income_grouped = data.groupby('debt').agg({'total_income': 'mean'})
print(income_grouped)

                       total_income
debt income_type                   
0    безработный      202722.000000
     госслужащий      169919.258206
     компаньон        199887.536565
     пенсионер        135145.954432
     предприниматель  499163.000000
     сотрудник        159978.793619
     студент           98201.000000
1    безработный       59956.000000
     в декрете         53829.000000
     госслужащий      152016.546512
     компаньон        193848.571809
     пенсионер        136385.074074
     сотрудник        155554.491989
       total_income
debt               
0     165660.432083
1     161158.393452


### Вывод

Мы видим, что в целом средний уровень дохода не сильно влияет на возврат кредита в срок. 

Однако если мы посмотрим на тип дохода, то увидим закономерность: чем ниже средний уровень дохода категорий 'безработный' и 'госслужащий', тем выше вероятность невозврата кредита. Особенно ярко это выражено у безработных (разница в средней величине дохода около 4 раз). 

- Как разные цели кредита влияют на его возврат в срок?

In [21]:
goals_grouped = data.groupby('purpose').agg({'debt': 'mean'})
print(goals_grouped.sort_values('debt'))

                  debt
purpose               
жилье         0.069043
недвижимость  0.074610
свадьба       0.079657
образование   0.092177
автомобиль    0.093547


### Вывод

Мы видим, что наиболее дисциплинированные клиенты в группе заемщиков на 'жилье'. Предположу, что это связано с закрытием базовой потребности и без крайней необходимости, люди старатся избегать долгов по ипотеке. Также можно заметить, что два близких понятия 'жилье' и 'недвижимость' имеют довольно сильное расхождение по среднему баллу наличия задолженности: с точки зрения семантики, можно выдвинуть гипотезы, что эти цели несколько отличны - возможно 'недвижимость' здесь фигурирует в несколько коммерческом контексте. Далее среднее значение растет, так как речь идет о значительно менее 'базовых' потребностях. При этом, можно предположить, что заемщики на свадьбу более дисциплинированны, так как в процессе возврата кредита используется помощь росдтвенников.

### Шаг 4. Общий вывод

Общий вывод относительно изначально озвученных целей исследования таков: 
1. Семейное положение влияет на возврат кредита в срок: женатые или разведенные более дисциплинированы, чем холостые. При этом гражданский брак практически сопоставим с холостыми. Самые дисциплинированные - вдовы/вдовцы, но эта категория не слишком многочисленна;
2. Количество детей влияет на возврат кредита в срок: с ростом количества детей, растет и средний балл по задолженности у клиентов. Вероятно, это связано с необходимостью погашать более важные затраты, связанные с содержанием детей.